# Chapter 32: Helm - Kubernetes Package Manager

While Chapter 31 explored Kubernetes Deployments for managing individual applications, production environments require orchestrating complex, multi-resource applications with environment-specific configurations. Helm—the Kubernetes package manager—addresses this complexity by templating Kubernetes manifests, managing release lifecycles, and enabling reusable, versioned packages called charts.

Helm transforms static YAML into dynamic, parameterized configurations, allowing teams to define applications once and deploy them across development, staging, and production with appropriate variations. Understanding Helm's templating engine, dependency management, and release tracking is essential for managing microservices architectures at scale.

## 32.1 Helm Architecture Overview

Helm operates as a client-side tool (Helm 3+) that interacts with the Kubernetes API server, managing packages through a layered architecture of charts, releases, and repositories.

### Core Concepts

**Charts:** Helm packages containing templated Kubernetes manifests, analogous to `.deb` or `.rpm` packages for operating systems. A chart includes:
- Template files for Kubernetes resources
- Default configuration values
- Metadata describing the chart and its dependencies
- Optional hooks for lifecycle automation

**Releases:** Instances of charts deployed to Kubernetes clusters. Each installation creates a release with:
- Unique release name (e.g., `production-api`, `staging-database`)
- Revision history tracking upgrades and rollbacks
- Computed values (user-provided merged with defaults)
- Deployed Kubernetes resources managed as a unit

**Repositories:** Collections of packaged charts distributed via HTTP servers, enabling sharing and versioning of applications.

### Helm 3 Architecture (Current)

Unlike Helm 2 which used Tiller (server-side component), Helm 3 is purely client-side:

```
┌─────────────────┐
│   Helm Client   │  (CLI tool on workstation/CI)
│   (Go binary)   │
└────────┬────────┘
         │ HTTPS
         ▼
┌─────────────────┐
│  Kubernetes API │  (kube-apiserver)
│    Server       │
└────────┬────────┘
         │
    ┌────┴────┐
    ▼         ▼
 Secrets   ConfigMaps  (Helm stores release state)
```

**Storage Mechanism:**
Helm stores release state in Kubernetes Secrets (default) or ConfigMaps within the deployment namespace, eliminating the security risks of Helm 2's Tiller while maintaining release history.

### Installation

```bash
# macOS
brew install helm

# Linux
curl https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3 | bash

# Verify installation
helm version --short

# Add stable repository
helm repo add stable https://charts.helm.sh/stable
helm repo add bitnami https://charts.bitnami.com/bitnami
helm repo update
```

## 32.2 Helm Charts Structure

A Helm chart is a directory containing structured files describing a Kubernetes application.

### Chart Directory Layout

```text
myapp/
├── Chart.yaml          # Chart metadata and dependencies
├── values.yaml         # Default configuration values
├── values.schema.json  # Optional: JSON schema for validation
├── charts/             # Sub-charts (dependencies)
│   └── postgresql-12.1.0.tgz
├── templates/          # Kubernetes manifest templates
│   ├── NOTES.txt       # Post-installation instructions
│   ├── _helpers.tpl    # Named template definitions
│   ├── deployment.yaml
│   ├── service.yaml
│   ├── ingress.yaml
│   ├── configmap.yaml
│   ├── secrets.yaml
│   ├── serviceaccount.yaml
│   ├── hpa.yaml        # Horizontal Pod Autoscaler
│   └── tests/          # Test pods for helm test
│       └── test-connection.yaml
├── crds/               # Custom Resource Definitions
│   └── myresource.yaml
└── README.md           # Documentation
└── LICENSE
```

### Chart.yaml Anatomy

```yaml
apiVersion: v2           # v2 for Helm 3, v1 for Helm 2
name: payment-service    # Chart name (lowercase, alphanumeric)
description: Payment processing microservice with PCI compliance
type: application        # application or library
version: 1.4.2           # Chart version (SemVer)
appVersion: "2.3.1"      # Application version (informational)
kubeVersion: ">=1.21.0-0" # Required Kubernetes version
icon: https://example.com/icon.png
home: https://github.com/company/payment-service
sources:
  - https://github.com/company/payment-service
maintainers:
  - name: Platform Team
    email: platform@company.com
keywords:
  - payments
  - financial
  - pci

# Dependencies (Helm 3)
dependencies:
  - name: postgresql
    version: "12.x.x"
    repository: "https://charts.bitnami.com/bitnami"
    condition: postgresql.enabled  # Only install if true
    tags:
      - databases
    alias: paymentdb               # Reference as paymentdb in templates
  
  - name: redis
    version: "17.x.x"
    repository: "https://charts.bitnami.com/bitnami"
    condition: redis.enabled

  - name: vault
    version: "0.24.x"
    repository: "https://helm.releases.hashicorp.com"
    import-values:                 # Import values from child to parent
      - child: defaults
        parent: vault.defaults
```

## 32.3 Creating Your First Chart

Generate a chart scaffold and customize for production use.

### Chart Generation

```bash
# Create new chart
helm create myapp

# Generated structure includes best-practice templates
tree myapp
```

### values.yaml (Configuration)

Define default configuration that users can override:

```yaml
# Global settings
global:
  imageRegistry: "registry.company.com"
  storageClass: "fast-ssd"
  securityContext:
    runAsNonRoot: true
    fsGroup: 1000

# Application configuration
replicaCount: 3

image:
  repository: payment-service
  tag: ""  # Defaults to Chart.appVersion if empty
  pullPolicy: IfNotPresent
  pullSecrets:
    - name: regcred

nameOverride: ""
fullnameOverride: ""

serviceAccount:
  create: true
  annotations: {}
  name: ""

# Security contexts
podSecurityContext:
  runAsNonRoot: true
  runAsUser: 1000
  runAsGroup: 1000
  fsGroup: 1000

securityContext:
  allowPrivilegeEscalation: false
  readOnlyRootFilesystem: true
  capabilities:
    drop:
      - ALL
  seccompProfile:
    type: RuntimeDefault

# Service configuration
service:
  type: ClusterIP
  port: 80
  targetPort: 8080
  annotations:
    prometheus.io/scrape: "true"
    prometheus.io/port: "8080"

# Ingress configuration
ingress:
  enabled: true
  className: nginx
  annotations:
    cert-manager.io/cluster-issuer: "letsencrypt-prod"
    nginx.ingress.kubernetes.io/rate-limit: "100"
  hosts:
    - host: api.company.com
      paths:
        - path: /payments
          pathType: Prefix
  tls:
    - secretName: api-tls
      hosts:
        - api.company.com

# Resource management
resources:
  limits:
    cpu: 1000m
    memory: 1Gi
  requests:
    cpu: 500m
    memory: 512Mi

# Autoscaling
autoscaling:
  enabled: true
  minReplicas: 3
  maxReplicas: 10
  targetCPUUtilizationPercentage: 80
  targetMemoryUtilizationPercentage: 80
  behavior:
    scaleUp:
      stabilizationWindowSeconds: 60
      policies:
        - type: Percent
          value: 100
          periodSeconds: 15
    scaleDown:
      stabilizationWindowSeconds: 300
      policies:
        - type: Percent
          value: 10
          periodSeconds: 60

# Pod Disruption Budget
pdb:
  enabled: true
  minAvailable: 2

# Health checks
livenessProbe:
  enabled: true
  path: /health/live
  initialDelaySeconds: 30
  periodSeconds: 10
  timeoutSeconds: 5
  failureThreshold: 3

readinessProbe:
  enabled: true
  path: /health/ready
  initialDelaySeconds: 5
  periodSeconds: 5
  timeoutSeconds: 3
  successThreshold: 2
  failureThreshold: 3

startupProbe:
  enabled: true
  path: /health/started
  initialDelaySeconds: 10
  periodSeconds: 5
  failureThreshold: 30

# Configuration and secrets
config:
  logLevel: info
  database:
    poolSize: 20
    timeout: 30s
  
secrets:
  apiKey: ""  # Override with --set or values file
  dbPassword: ""

# Node affinity
nodeSelector:
  node-type: worker

tolerations:
  - key: "dedicated"
    operator: "Equal"
    value: "payments"
    effect: "NoSchedule"

affinity:
  podAntiAffinity:
    preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 100
        podAffinityTerm:
          labelSelector:
            matchExpressions:
              - key: app.kubernetes.io/name
                operator: In
                values:
                  - myapp
          topologyKey: kubernetes.io/hostname

# Extra environment variables
extraEnv:
  - name: DD_AGENT_HOST
    valueFrom:
      fieldRef:
        fieldPath: status.hostIP
  - name: ENVIRONMENT
    value: production

# Sidecar containers (e.g., Istio proxy, Datadog agent)
sidecars:
  - name: istio-proxy
    image: istio/proxyv2:1.18.0
    resources:
      limits:
        cpu: 2000m
        memory: 1Gi

# Init containers
initContainers:
  - name: wait-for-db
    image: busybox:1.36
    command: ['sh', '-c', 'until nc -z paymentdb 5432; do sleep 2; done']
```

### Template Files

Templates use Go's `text/template` language with Helm's extensions (Sprig functions).

**templates/_helpers.tpl (Named templates):**

```yaml
{{/* Generate chart name */}}
{{- define "myapp.name" -}}
{{- default .Chart.Name .Values.nameOverride | trunc 63 | trimSuffix "-" }}
{{- end }}

{{/* Generate full name with release */}}
{{- define "myapp.fullname" -}}
{{- if .Values.fullnameOverride }}
{{- .Values.fullnameOverride | trunc 63 | trimSuffix "-" }}
{{- else }}
{{- $name := default .Chart.Name .Values.nameOverride }}
{{- if contains $name .Release.Name }}
{{- .Release.Name | trunc 63 | trimSuffix "-" }}
{{- else }}
{{- printf "%s-%s" .Release.Name $name | trunc 63 | trimSuffix "-" }}
{{- end }}
{{- end }}
{{- end }}

{{/* Chart label */}}
{{- define "myapp.chart" -}}
{{- printf "%s-%s" .Chart.Name .Chart.Version | replace "+" "_" | trunc 63 | trimSuffix "-" }}
{{- end }}

{{/* Common labels */}}
{{- define "myapp.labels" -}}
helm.sh/chart: {{ include "myapp.chart" . }}
{{ include "myapp.selectorLabels" . }}
{{- if .Chart.AppVersion }}
app.kubernetes.io/version: {{ .Chart.AppVersion | quote }}
{{- end }}
app.kubernetes.io/managed-by: {{ .Release.Service }}
app.kubernetes.io/part-of: {{ .Chart.Name }}
{{- end }}

{{/* Selector labels */}}
{{- define "myapp.selectorLabels" -}}
app.kubernetes.io/name: {{ include "myapp.name" . }}
app.kubernetes.io/instance: {{ .Release.Name }}
{{- end }}

{{/* Service account name */}}
{{- define "myapp.serviceAccountName" -}}
{{- if .Values.serviceAccount.create }}
{{- default (include "myapp.fullname" .) .Values.serviceAccount.name }}
{{- else }}
{{- default "default" .Values.serviceAccount.name }}
{{- end }}
{{- end }}

{{/* Generate container image path */}}
{{- define "myapp.image" -}}
{{- $registry := .Values.global.imageRegistry | default "" }}
{{- if $registry }}
{{- printf "%s/%s:%s" $registry .Values.image.repository (default .Chart.AppVersion .Values.image.tag) }}
{{- else }}
{{- printf "%s:%s" .Values.image.repository (default .Chart.AppVersion .Values.image.tag) }}
{{- end }}
{{- end }}
```

**templates/deployment.yaml:**

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ include "myapp.fullname" . }}
  labels:
    {{- include "myapp.labels" . | nindent 4 }}
  annotations:
    # Rollout restart on config change
    checksum/config: {{ include (print $.Template.BasePath "/configmap.yaml") . | sha256sum }}
    checksum/secrets: {{ include (print $.Template.BasePath "/secrets.yaml") . | sha256sum }}
spec:
  {{- if not .Values.autoscaling.enabled }}
  replicas: {{ .Values.replicaCount }}
  {{- end }}
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: {{ .Values.maxSurge | default "25%" }}
      maxUnavailable: {{ .Values.maxUnavailable | default "0" }}
  selector:
    matchLabels:
      {{- include "myapp.selectorLabels" . | nindent 6 }}
  template:
    metadata:
      annotations:
        # Force redeployment when config changes
        checksum/config: {{ include (print $.Template.BasePath "/configmap.yaml") . | sha256sum }}
        prometheus.io/scrape: "true"
        prometheus.io/port: "8080"
      labels:
        {{- include "myapp.selectorLabels" . | nindent 8 }}
    spec:
      {{- with .Values.imagePullSecrets }}
      imagePullSecrets:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      
      serviceAccountName: {{ include "myapp.serviceAccountName" . }}
      securityContext:
        {{- toYaml .Values.podSecurityContext | nindent 8 }}
      
      {{- if .Values.initContainers }}
      initContainers:
        {{- toYaml .Values.initContainers | nindent 8 }}
      {{- end }}
      
      containers:
        - name: {{ .Chart.Name }}
          securityContext:
            {{- toYaml .Values.securityContext | nindent 12 }}
          image: {{ include "myapp.image" . }}
          imagePullPolicy: {{ .Values.image.pullPolicy }}
          
          ports:
            - name: http
              containerPort: {{ .Values.service.targetPort }}
              protocol: TCP
          
          env:
            - name: POD_NAME
              valueFrom:
                fieldRef:
                  fieldPath: metadata.name
            - name: POD_NAMESPACE
              valueFrom:
                fieldRef:
                  fieldPath: metadata.namespace
            {{- range .Values.extraEnv }}
            - name: {{ .name }}
              value: {{ .value | quote }}
            {{- end }}
          
          envFrom:
            - configMapRef:
                name: {{ include "myapp.fullname" . }}-config
            - secretRef:
                name: {{ include "myapp.fullname" . }}-secrets
                optional: true
          
          {{- if .Values.livenessProbe.enabled }}
          livenessProbe:
            httpGet:
              path: {{ .Values.livenessProbe.path }}
              port: http
            initialDelaySeconds: {{ .Values.livenessProbe.initialDelaySeconds }}
            periodSeconds: {{ .Values.livenessProbe.periodSeconds }}
            timeoutSeconds: {{ .Values.livenessProbe.timeoutSeconds }}
            failureThreshold: {{ .Values.livenessProbe.failureThreshold }}
          {{- end }}
          
          {{- if .Values.readinessProbe.enabled }}
          readinessProbe:
            httpGet:
              path: {{ .Values.readinessProbe.path }}
              port: http
            initialDelaySeconds: {{ .Values.readinessProbe.initialDelaySeconds }}
            periodSeconds: {{ .Values.readinessProbe.periodSeconds }}
            timeoutSeconds: {{ .Values.readinessProbe.timeoutSeconds }}
            successThreshold: {{ .Values.readinessProbe.successThreshold }}
            failureThreshold: {{ .Values.readinessProbe.failureThreshold }}
          {{- end }}
          
          resources:
            {{- toYaml .Values.resources | nindent 12 }}
          
          volumeMounts:
            - name: tmp
              mountPath: /tmp
            {{- if .Values.persistence.enabled }}
            - name: data
              mountPath: {{ .Values.persistence.mountPath }}
            {{- end }}
      
      {{- if .Values.sidecars }}
        {{- toYaml .Values.sidecars | nindent 8 }}
      {{- end }}
      
      volumes:
        - name: tmp
          emptyDir: {}
        {{- if .Values.persistence.enabled }}
        - name: data
          persistentVolumeClaim:
            claimName: {{ .Values.persistence.existingClaim | default (include "myapp.fullname" .) }}
        {{- end }}
      
      {{- with .Values.nodeSelector }}
      nodeSelector:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      
      {{- with .Values.affinity }}
      affinity:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      
      {{- with .Values.tolerations }}
      tolerations:
        {{- toYaml . | nindent 8 }}
      {{- end }}
```

**templates/service.yaml:**

```yaml
apiVersion: v1
kind: Service
metadata:
  name: {{ include "myapp.fullname" . }}
  labels:
    {{- include "myapp.labels" . | nindent 4 }}
  annotations:
    {{- with .Values.service.annotations }}
    {{- toYaml . | nindent 4 }}
    {{- end }}
spec:
  type: {{ .Values.service.type }}
  ports:
    - port: {{ .Values.service.port }}
      targetPort: http
      protocol: TCP
      name: http
  selector:
    {{- include "myapp.selectorLabels" . | nindent 4 }}
```

**templates/hpa.yaml (Horizontal Pod Autoscaler):**

```yaml
{{- if .Values.autoscaling.enabled }}
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: {{ include "myapp.fullname" . }}
  labels:
    {{- include "myapp.labels" . | nindent 4 }}
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: {{ include "myapp.fullname" . }}
  minReplicas: {{ .Values.autoscaling.minReplicas }}
  maxReplicas: {{ .Values.autoscaling.maxReplicas }}
  metrics:
    {{- if .Values.autoscaling.targetCPUUtilizationPercentage }}
    - type: Resource
      resource:
        name: cpu
        target:
          type: Utilization
          averageUtilization: {{ .Values.autoscaling.targetCPUUtilizationPercentage }}
    {{- end }}
    {{- if .Values.autoscaling.targetMemoryUtilizationPercentage }}
    - type: Resource
      resource:
        name: memory
        target:
          type: Utilization
          averageUtilization: {{ .Values.autoscaling.targetMemoryUtilizationPercentage }}
    {{- end }}
  behavior:
    {{- toYaml .Values.autoscaling.behavior | nindent 4 }}
{{- end }}
```

## 32.4 Templates and Values

Helm's templating engine transforms values into Kubernetes manifests using Go templates with the Sprig function library.

### Built-in Objects

Templates access these top-level objects:

- `.Values`: Values from values.yaml and user overrides
- `.Release`: Release information (Name, Namespace, Revision, IsUpgrade, IsInstall)
- `.Chart`: Chart.yaml contents (Name, Version, AppVersion, etc.)
- `.Capabilities`: Cluster capabilities (API versions, Kubernetes version)
- `.Template`: Current template information (Name, BasePath)
- `.Files`: Access to non-template files in chart

**Example usage:**
```yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: {{ .Release.Name }}-config
  namespace: {{ .Release.Namespace }}
data:
  version: {{ .Chart.Version }}
  deployedAt: {{ now | htmlDate }}
  k8sVersion: {{ .Capabilities.KubeVersion.Version }}
```

### Template Functions

Helm includes over 60 functions from the Sprig library:

**String Functions:**
```yaml
# Default values
image: {{ .Values.image.tag | default "latest" }}

# Uppercase/lowercase
env: {{ .Values.environment | upper }}

# Replace
name: {{ .Values.name | replace "_" "-" }}

# Truncate
name: {{ .Values.longName | trunc 63 | trimSuffix "-" }}
```

**Mathematical Functions:**
```yaml
replicas: {{ .Values.replicaCount | add 2 }}
memory: {{ .Values.memoryGB | mul 1024 }}Mi
```

**Date Functions:**
```yaml
timestamp: {{ now | quote }}
formatted: {{ now | date "2006-01-02" }}
```

**Encoding:**
```yaml
# Base64 encode for Secrets
password: {{ .Values.password | b64enc }}

# Generate random
password: {{ randAlphaNum 20 | b64enc | quote }}
```

**Control Structures:**
```yaml
# If/else
{{- if .Values.ingress.enabled }}
# ... ingress spec ...
{{- else if .Values.route.enabled }}
# ... OpenShift route ...
{{- else }}
# No ingress
{{- end }}

# With (set scope)
{{- with .Values.resources }}
resources:
  limits:
    cpu: {{ .limits.cpu }}
{{- end }}

# Range (iteration)
env:
{{- range $key, $value := .Values.env }}
  - name: {{ $key }}
    value: {{ $value | quote }}
{{- end }}

# Range over list
{{- range .Values.secrets }}
- name: {{ .name }}
  valueFrom:
    secretKeyRef:
      name: {{ .secretName }}
      key: {{ .key }}
{{- end }}
```

### Variable Definitions

Define variables for complex logic:

```yaml
{{- $fullname := include "myapp.fullname" . -}}
{{- $replicas := .Values.replicaCount | int -}}
{{- $hasCredentials := and .Values.username .Values.password -}}

apiVersion: v1
kind: Secret
metadata:
  name: {{ $fullname }}-credentials
{{- if $hasCredentials }}
stringData:
  username: {{ .Values.username }}
  password: {{ .Values.password }}
{{- else }}
data: {}
{{- end }}
```

## 32.5 Chart Dependencies

Complex applications require databases, caches, or other services. Helm manages these through chart dependencies.

### Declaring Dependencies

```yaml
# Chart.yaml
dependencies:
  - name: postgresql
    version: "12.x.x"
    repository: "https://charts.bitnami.com/bitnami"
    condition: postgresql.enabled  # Only if .Values.postgresql.enabled == true
    alias: db                      # Referenced as .Values.db in parent
    tags:
      - databases
  
  - name: redis
    version: "17.x.x"
    repository: "https://charts.bitnami.com/bitnami"
    condition: redis.enabled
  
  - name: kafka
    version: "22.x.x"
    repository: "https://charts.bitnami.com/bitnami"
    import-values:                 # Import child values to parent scope
      - child: defaults
        parent: kafka.defaults
```

### Dependency Management Commands

```bash
# Download dependencies to charts/ directory
helm dependency update

# Check for outdated dependencies
helm dependency list

# Verify dependencies exist
helm dependency build  # Fail if charts/ missing or wrong version
```

### Conditional Dependencies

```yaml
# values.yaml
postgresql:
  enabled: true  # Set to false to disable
  auth:
    username: paymentuser
    password: changeme
    database: payments
  primary:
    persistence:
      enabled: true
      storageClass: "fast-ssd"
      size: 10Gi

redis:
  enabled: false  # Not needed for this deployment
```

**Using dependency services in templates:**
```yaml
# Reference the aliased dependency
env:
  - name: DB_HOST
    value: {{ .Release.Name }}-db  # Uses alias "db"
  - name: DB_PORT
    value: "5432"
```

### Subcharts and Global Values

Values prefixed with `global.` are accessible to parent and all children:

```yaml
# values.yaml
global:
  imageRegistry: "registry.company.com"
  storageClass: "ssd"
  environment: "production"

postgresql:
  image:
    registry: "{{ .Values.global.imageRegistry }}"
```

## 32.6 Helm Hooks

Hooks execute Kubernetes jobs at specific lifecycle points, enabling database migrations, backups, or notifications.

### Hook Types

| Hook | Execution Point |
|------|----------------|
| `pre-install` | After templates render, before resources created |
| `post-install` | After all resources created and ready |
| `pre-delete` | Before resources deleted |
| `post-delete` | After resources deleted |
| `pre-upgrade` | Before upgrade modifies resources |
| `post-upgrade` | After upgrade completes |
| `pre-rollback` | Before rollback executes |
| `post-rollback` | After rollback completes |
| `test` | When `helm test` runs |

### Hook Implementation

```yaml
# templates/hooks/db-migrate.yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: {{ include "myapp.fullname" . }}-db-migrate
  annotations:
    "helm.sh/hook": pre-upgrade,pre-install  # Run before install and upgrade
    "helm.sh/hook-weight": "1"               # Execution order (low first)
    "helm.sh/hook-delete-policy": hook-succeeded  # Cleanup after success
spec:
  template:
    spec:
      restartPolicy: Never
      containers:
        - name: db-migrate
          image: "{{ include "myapp.image" . }}"
          command: ["npm", "run", "migrate:up"]
          env:
            - name: DATABASE_URL
              valueFrom:
                secretKeyRef:
                  name: {{ include "myapp.fullname" . }}-db
                  key: url
```

### Hook Weights and Ordering

```yaml
# hooks/pre-install-1-secrets.yaml
metadata:
  annotations:
    "helm.sh/hook": pre-install
    "helm.sh/hook-weight": "-5"  # Runs first (negative weight)

# hooks/pre-install-2-configmap.yaml  
metadata:
  annotations:
    "helm.sh/hook": pre-install
    "helm.sh/hook-weight": "0"   # Default, runs after -5

# hooks/pre-install-3-job.yaml
metadata:
  annotations:
    "helm.sh/hook": pre-install
    "helm.sh/hook-weight": "5"   # Runs last
```

### Database Migration Best Practices

```yaml
# 1. Pre-install/upgrade: Run migrations
# templates/hooks/migration.yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: {{ include "myapp.fullname" . }}-migrate
  annotations:
    "helm.sh/hook": pre-install,pre-upgrade
    "helm.sh/hook-weight": "0"
    "helm.sh/hook-delete-policy": before-hook-creation,hook-succeeded
spec:
  activeDeadlineSeconds: 600
  template:
    spec:
      restartPolicy: Never
      containers:
      - name: migrate
        image: "{{ include "myapp.image" . }}"
        command: ["./scripts/migrate.sh"]
        envFrom:
        - secretRef:
            name: {{ include "myapp.fullname" . }}-db-credentials

# 2. Post-install/upgrade: Verify application health
# templates/hooks/smoke-test.yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: {{ include "myapp.fullname" . }}-smoke-test
  annotations:
    "helm.sh/hook": post-install,post-upgrade
    "helm.sh/hook-weight": "0"
spec:
  template:
    spec:
      restartPolicy: Never
      containers:
      - name: test
        image: curlimages/curl:latest
        command:
        - sh
        - -c
        - |
          sleep 10
          curl -sf http://{{ include "myapp.fullname" . }}/health || exit 1
          echo "Smoke test passed"
```

## 32.7 CI/CD Integration with Helm

Helm charts are typically built and deployed through CI/CD pipelines.

### Chart Packaging and Publishing

```yaml
# GitLab CI example
stages:
  - build
  - package
  - publish

build:image:
  stage: build
  script:
    - docker build -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA .

package:chart:
  stage: package
  image: alpine/helm:latest
  script:
    # Update chart version to match Git tag or commit
    - helm package . --version $(cat VERSION) --app-version $CI_COMMIT_SHA
    
    # Lint chart
    - helm lint .
    
    # Test template rendering
    - helm template myapp . --values values-production.yaml > rendered.yaml
    
    # Validate against Kubernetes API
    - kubectl apply --dry-run=client -f rendered.yaml
  artifacts:
    paths:
      - myapp-*.tgz

publish:chart:
  stage: publish
  image: curlimages/curl:latest
  script:
    # Publish to ChartMuseum or Harbor
    - |
      curl --data-binary "@myapp-$(cat VERSION).tgz" \
        $CHART_REPO_URL/api/charts \
        -u $CHART_REPO_USER:$CHART_REPO_PASS
  only:
    - tags
```

### GitOps Deployment with Helm

Using Helm with ArgoCD or Flux:

```yaml
# ArgoCD Application
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: payment-service
  namespace: argocd
spec:
  project: production
  source:
    repoURL: https://charts.company.com
    chart: payment-service
    targetRevision: 1.4.2
    helm:
      valueFiles:
        - values-production.yaml
      parameters:
        - name: image.tag
          value: v2.3.1
        - name: replicaCount
          value: "5"
  destination:
    server: https://kubernetes.default.svc
    namespace: payments
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
    syncOptions:
      - CreateNamespace=true
```

### Helmfile (Declarative Helm Management)

For managing multiple charts:

```yaml
# helmfile.yaml
releases:
  - name: ingress-nginx
    namespace: ingress
    chart: ingress-nginx/ingress-nginx
    version: 4.7.0
    values:
      - values/ingress-nginx.yaml
  
  - name: cert-manager
    namespace: cert-manager
    chart: jetstack/cert-manager
    version: 1.12.0
    values:
      - values/cert-manager.yaml
  
  - name: payment-service
    namespace: payments
    chart: ./charts/payment-service
    version: 1.4.2
    values:
      - values/payment-service/production.yaml
      - values/payment-service/secrets.yaml
    set:
      - name: image.tag
        value: {{ requiredEnv "IMAGE_TAG" }}
    hooks:
      - events: ["prepare"]
        showlogs: true
        command: "kubectl"
        args: ["create", "namespace", "payments", "--dry-run=client", "-o", "yaml", "|", "kubectl", "apply", "-f", "-"]
```

## 32.8 Helm 3 vs Helm 2

Helm 3 removed the server-side Tiller component, improving security and simplifying architecture.

### Key Differences

| Feature | Helm 2 | Helm 3 |
|---------|--------|--------|
| **Architecture** | Client-Server (Tiller) | Client-only |
| **Security** | Tiller runs with cluster-admin | Uses user kubeconfig permissions |
| **Release Storage** | ConfigMaps in kube-system | Secrets in release namespace |
| **Libraries** | Not supported | Chart type: library |
| **JSON Schema** | Not supported | values.schema.json validation |
| **Release Name** | Cluster-scoped | Namespace-scoped |
| **Requirements** | requirements.yaml | Chart.yaml dependencies |

### Migration from Helm 2

```bash
# Install helm-3-binary as helm3
# Convert Helm 2 releases to Helm 3
helm3 2to3 convert my-release

# Cleanup Helm 2 release data (after verification)
helm3 2to3 cleanup

# Uninstall Helm 2 Tiller
helm reset --force
```

## 32.9 Best Practices

### Security

**1. RBAC Integration:**
Helm 3 uses the user's Kubernetes credentials—ensure CI service accounts have minimal required permissions.

**2. Secret Management:**
Never commit secrets to values.yaml. Use:
- External secret operators (External Secrets, Vault)
- Sealed Secrets
- Helm Secrets plugin with SOPS

**3. Chart Verification:**
```bash
# Sign charts with PGP
helm package --sign --key 'Platform Team' ./myapp
helm verify myapp-1.0.0.tgz

# Check provenance
helm install --verify myapp ./myapp-1.0.0.tgz
```

### Chart Development

**1. Semantic Versioning:**
- MAJOR: Breaking changes (API changes, incompatible schema)
- MINOR: New features, backward compatible
- PATCH: Bug fixes

**2. Immutability:**
Never modify published chart versions—publish new versions instead.

**3. Testing:**
```bash
# Lint
helm lint .

# Unit tests (helm-unittest plugin)
helm unittest .

# Integration tests
helm install --dry-run --debug myapp . --values test-values.yaml
```

### Production Readiness

**1. Resource Limits:**
Always specify requests and limits in values.yaml.

**2. Pod Disruption Budgets:**
Include PDB templates for high availability.

**3. Graceful Shutdown:**
Configure terminationGracePeriodSeconds and preStop hooks.

**4. Health Checks:**
Standardized liveness, readiness, and startup probes.

---

## Chapter Summary and Preview

In this chapter, we explored Helm as the Kubernetes package manager, transforming static manifests into dynamic, reusable, and versioned applications. We examined the chart structure comprising templates, values, and metadata, enabling a single chart to deploy across development, staging, and production through environment-specific value files. The templating engine—leveraging Go templates with Sprig functions—provides powerful logic for generating Kubernetes resources dynamically while maintaining readability. Chart dependencies enable composing complex applications from modular components (databases, caches) with conditional installation and value importing. Helm hooks facilitate lifecycle automation for database migrations, smoke tests, and cleanup operations executed at precise installation or upgrade phases. We detailed CI/CD integration patterns for packaging, linting, and publishing charts to repositories, alongside GitOps deployment strategies using ArgoCD. The transition from Helm 2 to Helm 3 eliminated the security risk of Tiller by adopting a client-only architecture using Kubernetes Secrets for state storage and user RBAC permissions for authorization. Best practices emphasized semantic versioning, immutability of published charts, secret management through external operators, and comprehensive resource definitions including Pod Disruption Budgets and health probes for production resilience.

**Key Takeaways:**
- Never use Helm 2's Tiller in production environments; migrate to Helm 3 immediately to eliminate cluster-admin privilege requirements and security exposure
- Implement chart testing through `helm lint`, `helm template --validate`, and the helm-unittest plugin to catch rendering errors before deployment
- Use hooks (pre-install, pre-upgrade) for database schema migrations, but ensure idempotency and include rollback procedures; never use hooks for long-running jobs without timeout configurations
- Store chart dependencies in version control (after `helm dependency update`) to ensure reproducible builds, or use `helm dependency build` in CI to verify lockfile consistency
- Separate configuration from code using values files (values-production.yaml, values-staging.yaml) rather than overriding individual values with `--set`, which is error-prone and non-auditable
- Enable JSON schema validation (values.schema.json) to prevent deployment failures from typos or invalid value types, catching errors at the client side before Kubernetes API submission

**Next Chapter Preview:**
Chapter 33: Advanced Helm explores sophisticated patterns including subchart development, library charts for reusable template logic, advanced templating with functions and pipelines, and the Helmfile tool for managing multi-chart deployments. We will examine chart testing strategies using the helm-test framework, security hardening with signed charts and provenance files, and integration with external secret management systems. This chapter builds upon the foundational Helm concepts to enable enterprise-grade chart development and complex application lifecycle management across multiple Kubernetes clusters and environments.